# Trend Following Strategies**QuantLearn Module**: Strategy Types**Difficulty**: Intermediate**Time**: ~25 minutesLearn to build and backtest trend-following strategies using moving averages, breakouts, and momentum signals.

In [ ]:
#@title 📦 Setupimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltnp.random.seed(42)plt.style.use('seaborn-v0_8-whitegrid')# Generate trending price data with regimesdef generate_trending_data(n_days=500):    dates = pd.date_range('2022-01-01', periods=n_days, freq='B')    # Create regime-switching returns    returns = []    regime = 1  # Start bullish    for i in range(n_days):        if np.random.random() < 0.02:  # 2% chance of regime switch            regime *= -1        drift = 0.001 * regime  # Positive or negative drift        ret = np.random.normal(drift, 0.015)        returns.append(ret)    prices = 100 * np.cumprod(1 + np.array(returns))    return pd.DataFrame({'Date': dates, 'Price': prices, 'Return': returns}).set_index('Date')df = generate_trending_data()print("✓ Setup complete!")print(f"Generated {len(df)} days of price data")

## 1. Moving Average CrossoverThe classic trend-following approach:- **Fast MA** (e.g., 20-day) reacts quickly to price changes- **Slow MA** (e.g., 50-day) represents the longer-term trend- **Signal**: Go long when fast > slow, short when fast < slow

In [ ]:
# MA Crossover Strategyfast_period = 20slow_period = 50df['MA_Fast'] = df['Price'].rolling(fast_period).mean()df['MA_Slow'] = df['Price'].rolling(slow_period).mean()# Signal: 1 = long, -1 = shortdf['MA_Signal'] = np.where(df['MA_Fast'] > df['MA_Slow'], 1, -1)df['MA_Position'] = df['MA_Signal'].shift(1)df['MA_Return'] = df['MA_Position'] * df['Return']# Visualizefig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)axes[0].plot(df['Price'], alpha=0.7, label='Price')axes[0].plot(df['MA_Fast'], label=f'{fast_period}-day MA')axes[0].plot(df['MA_Slow'], label=f'{slow_period}-day MA')axes[0].set_ylabel('Price')axes[0].legend()axes[0].set_title('Moving Average Crossover Strategy')# Cumulative returnsdf_clean = df.dropna()cum_market = (1 + df_clean['Return']).cumprod()cum_strategy = (1 + df_clean['MA_Return']).cumprod()axes[1].plot(cum_market, label='Buy & Hold', alpha=0.7)axes[1].plot(cum_strategy, label='MA Crossover', linewidth=2)axes[1].set_ylabel('Cumulative Return')axes[1].legend()plt.tight_layout()plt.show()

## 2. Breakout StrategyTrade when price breaks above/below recent high/low:- **Donchian Channel**: N-day high and low- **Breakout signal**: Long on new high, short on new low

In [ ]:
# Breakout Strategy (Donchian Channel)lookback = 20df['High_N'] = df['Price'].rolling(lookback).max()df['Low_N'] = df['Price'].rolling(lookback).min()# Signal: breakout above high = long, below low = shortdf['Breakout_Signal'] = 0df.loc[df['Price'] >= df['High_N'].shift(1), 'Breakout_Signal'] = 1df.loc[df['Price'] <= df['Low_N'].shift(1), 'Breakout_Signal'] = -1# Forward fill signal (hold position)df['Breakout_Signal'] = df['Breakout_Signal'].replace(0, np.nan).ffill().fillna(0)df['Breakout_Position'] = df['Breakout_Signal'].shift(1)df['Breakout_Return'] = df['Breakout_Position'] * df['Return']# Plotfig, ax = plt.subplots(figsize=(14, 5))ax.plot(df['Price'], label='Price', alpha=0.7)ax.plot(df['High_N'], '--', color='green', alpha=0.5, label=f'{lookback}-day High')ax.plot(df['Low_N'], '--', color='red', alpha=0.5, label=f'{lookback}-day Low')ax.legend()ax.set_title('Donchian Channel Breakout')plt.show()

## 3. Momentum StrategyTrade based on recent performance:- Calculate N-day momentum (cumulative return)- Long if momentum > 0, short if < 0

In [ ]:
# Momentum Strategymomentum_period = 20df['Momentum'] = df['Price'].pct_change(momentum_period)df['Mom_Signal'] = np.where(df['Momentum'] > 0, 1, -1)df['Mom_Position'] = df['Mom_Signal'].shift(1)df['Mom_Return'] = df['Mom_Position'] * df['Return']# Compare all strategiesdf_compare = df.dropna()strategies = {    'Buy & Hold': (1 + df_compare['Return']).cumprod(),    'MA Crossover': (1 + df_compare['MA_Return']).cumprod(),    'Breakout': (1 + df_compare['Breakout_Return']).cumprod(),    'Momentum': (1 + df_compare['Mom_Return']).cumprod()}plt.figure(figsize=(14, 6))for name, cum_ret in strategies.items():    plt.plot(cum_ret, label=name, linewidth=2 if name != 'Buy & Hold' else 1)plt.ylabel('Cumulative Return')plt.title('Trend Following Strategy Comparison')plt.legend()plt.show()

## Exercise: Build a Combined Trend SignalCreate a strategy that combines multiple trend signals:1. Go long only when ALL signals agree (MA, Breakout, Momentum all positive)2. Go short only when ALL signals agree (all negative)3. Stay flat when signals disagree

In [ ]:
# Exercise: Combined trend signal# TODO: Create combined signal# Hint: Sum the three signals, only trade when |sum| == 3df['Combined_Signal'] = None  # Your code here# TODO: Calculate strategy returnsdf['Combined_Position'] = Nonedf['Combined_Return'] = None# Compare to individual strategies# ...

In [ ]:
#@title 💡 Solution# Sum signals: only trade when all 3 agreesignal_sum = df['MA_Signal'] + df['Breakout_Signal'].fillna(0) + df['Mom_Signal']df['Combined_Signal'] = 0df.loc[signal_sum == 3, 'Combined_Signal'] = 1   # All bullishdf.loc[signal_sum == -3, 'Combined_Signal'] = -1  # All bearish# Otherwise stay flat (0)df['Combined_Position'] = df['Combined_Signal'].shift(1)df['Combined_Return'] = df['Combined_Position'] * df['Return']# Plotdf_final = df.dropna()plt.figure(figsize=(14, 6))plt.plot((1 + df_final['Return']).cumprod(), label='Buy & Hold', alpha=0.7)plt.plot((1 + df_final['MA_Return']).cumprod(), label='MA Crossover', alpha=0.7)plt.plot((1 + df_final['Combined_Return']).cumprod(), label='Combined Signal', linewidth=2)plt.ylabel('Cumulative Return')plt.title('Combined Trend Signal Performance')plt.legend()plt.show()# Statsprint("Combined Strategy Stats:")combined_rets = df_final['Combined_Return']print(f"Annual Return: {combined_rets.mean() * 252 * 100:.1f}%")print(f"Annual Vol: {combined_rets.std() * np.sqrt(252) * 100:.1f}%")print(f"Sharpe: {combined_rets.mean() / combined_rets.std() * np.sqrt(252):.2f}")print(f"Time in Market: {(df_final['Combined_Signal'] != 0).mean() * 100:.0f}%")

## Summary| Strategy | Description | Pros | Cons ||----------|-------------|------|------|| MA Crossover | Fast MA vs Slow MA | Simple, always in market | Whipsaws in sideways markets || Breakout | Trade new highs/lows | Catches big moves | Many false breakouts || Momentum | Recent return direction | Captures trends | Vulnerable to reversals || Combined | Require agreement | Fewer trades, higher conviction | May miss opportunities |**Key insight**: Trend following works in trending markets but suffers during range-bound periods. Consider regime filtering!**Next**: Mean Reversion strategies.